In [1]:
import pandas as pd
import numpy as np
from orm_sqlalchemy.index import DatabaseHandler
from utils.utils import clean_string

In [2]:
db = DatabaseHandler()
db.create_tables(schema_name='trusted')
query_result = db.execute_query('CREATE SCHEMA IF NOT EXISTS trusted')


In [3]:
listings_result = db.execute_query("SELECT * FROM raw.listings")
df_listings = pd.DataFrame(listings_result.fetchall(), columns=listings_result.keys())

In [4]:
for col in df_listings.columns:
    print(f"Coluna {col:<50} - type: {str(df_listings[col].dtypes):<10} - qtde de nan: {df_listings[col].isna().sum()}")


Coluna id                                                 - type: int64      - qtde de nan: 0
Coluna listing_url                                        - type: object     - qtde de nan: 0
Coluna scrape_id                                          - type: int64      - qtde de nan: 0
Coluna last_scraped                                       - type: object     - qtde de nan: 0
Coluna source                                             - type: object     - qtde de nan: 0
Coluna name                                               - type: object     - qtde de nan: 0
Coluna description                                        - type: object     - qtde de nan: 27
Coluna neighborhood_overview                              - type: object     - qtde de nan: 785
Coluna picture_url                                        - type: object     - qtde de nan: 0
Coluna host_id                                            - type: int64      - qtde de nan: 0
Coluna host_url                                          

In [5]:
# Pela análise, foi identificado que há 3 linhas com quantidades significativas de NaN
df_listings[
    (df_listings['host_name'].isna()) &
    (df_listings['host_thumbnail_url'].isna()) &
    (df_listings['host_picture_url'].isna()) &
    (df_listings['host_since'].isna()) &
    (df_listings['host_listings_count'].isna()) &
    (df_listings['host_total_listings_count'].isna()) &
    (df_listings['host_has_profile_pic'].isna()) &
    (df_listings['host_identity_verified'].isna()) &
    (df_listings['host_verifications'].isna())
]
# 9 colunas com valor NaN

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month


In [6]:
# remoção dos 3 registros
df_listings.drop(df_listings[df_listings['host_name'].isna()].index, inplace=True)
for col in df_listings.columns:
    print(f"Coluna {col:<50} - type: {str(df_listings[col].dtypes):<10} - qtde de nan: {df_listings[col].isna().sum()}")

Coluna id                                                 - type: int64      - qtde de nan: 0
Coluna listing_url                                        - type: object     - qtde de nan: 0
Coluna scrape_id                                          - type: int64      - qtde de nan: 0
Coluna last_scraped                                       - type: object     - qtde de nan: 0
Coluna source                                             - type: object     - qtde de nan: 0
Coluna name                                               - type: object     - qtde de nan: 0
Coluna description                                        - type: object     - qtde de nan: 27
Coluna neighborhood_overview                              - type: object     - qtde de nan: 785
Coluna picture_url                                        - type: object     - qtde de nan: 0
Coluna host_id                                            - type: int64      - qtde de nan: 0
Coluna host_url                                          

### Outros tratamentos de NaN

<table>
    <thead>
        <tr>
            <td>Coluna</td>
            <td>Tipo de dado</td>
            <td>Quantidade de NaN</td>
            <td>Dicionário</td>
            <td>Valor inserido</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>description</td>
            <td>Object</td>
            <td>659/31961</td>
            <td>Descrição geral do imóvel</td>
            <td>"No description provided"</td>
        </tr>
        <tr>
            <td>neighborhood_overview</td>
            <td>Object</td>
            <td>15223/31961</td>
            <td>Descrição da vizinhança</td>
            <td>"No neighborhood overview provided"</td>
        </tr>
        <tr>
            <td>host_location</td>
            <td>Object</td>
            <td>6209/31961</td>
            <td>Localização do anfitrião (host)</td>
            <td>"Rio de Janeiro, Brazil"</td>
        </tr>
        <tr>
            <td>host_about</td>
            <td>Object</td>
            <td>16118/31961</td>
            <td>Descrição do do anfitrião (host)</td>
            <td>"No host description provided"</td>
        </tr>
        <tr>
            <td>host_is_superhost</td>
            <td>Object</td>
            <td>656/31961</td>
            <td>Flag "Superhost" fornecida pelo AirBnB</td>
            <td>"f"</td>
        </tr>
        <tr>
            <td>host_response_time</td>
            <td>Object</td>
            <td>6705/31961</td>
            <td>Tempo de resposta anfitrião</td>
            <td>"a few days or more" (a mais demorada possível)</td>
        </tr>
        <tr>
            <td>host_response_rate</td>
            <td>Object</td>
            <td>6705/31961</td>
            <td>Taxa de resposta do anfitrião</td>
            <td>"0%"</td>
        </tr>
        <tr>
            <td>host_neighbourhood</td>
            <td>Object</td>
            <td>8059/31961</td>
            <td>Vizinhança do anfitrião</td>
            <td>"0%"</td>
        </tr>
        <tr>
            <td>bathrooms_text</td>
            <td>Object</td>
            <td>28/31961</td>
            <td>Quantidade de banheiros</td>
            <td>"0 shared baths"</td>
        </tr>
        <tr>
            <td>bedrooms</td>
            <td>float64</td>
            <td>7347/31961</td>
            <td>Quantidade de quartos</td>
            <td>0</td>
        </tr>
        <tr>
            <td>beds</td>
            <td>float64</td>
            <td>175/31961</td>
            <td>Quantidade de camas</td>
            <td>0</td>
        </tr>
        <tr>
            <td>host_acceptance_rate</td>
            <td>Object</td>
            <td>4778/31961</td>
            <td>Taxa de aceitação do anfitrião</td>
            <td>"0%"</td>
        </tr>
    </tbody>
</table>

In [7]:
df_listings['description'].fillna("No description provided", inplace=True)
df_listings['neighborhood_overview'].fillna("No neighborhood overview provided", inplace=True)
df_listings['host_location'].fillna("Rio de Janeiro, Brazil", inplace=True)
df_listings['host_about'].fillna("No host description provided", inplace=True)
df_listings['host_is_superhost'].fillna('f', inplace=True)
df_listings['host_response_time'].fillna('a few days or more', inplace=True)
df_listings['host_response_rate'].fillna('0%', inplace=True)
df_listings['host_neighbourhood'].fillna(df_listings['host_location'], inplace=True)
df_listings['bathrooms_text'].fillna('0 shared baths', inplace=True)
df_listings['bedrooms'].fillna(0, inplace=True)
df_listings['beds'].fillna(0, inplace=True)
df_listings['host_acceptance_rate'].fillna('0%', inplace=True)

In [8]:
# Não tratados

# Coluna review_scores_accuracy - type: float64 - qtde de nan: 8294
# Coluna review_scores_cleanliness - type: float64 - qtde de nan: 8293
# Coluna review_scores_checkin - type: float64 - qtde de nan: 8293
# Coluna review_scores_communication - type: float64 - qtde de nan: 8294
# Coluna review_scores_location - type: float64 - qtde de nan: 8296
# Coluna review_scores_value - type: float64 - qtde de nan: 8294

### Colunas especiais - avaliações (reviews) inexistentes

1. Coluna first_review - type: object - qtde de nan: 8137
> Data da primeira avaliação

2. Coluna last_review - type: object - qtde de nan: 8137
> Data da última avaliação

3. Coluna review_scores_rating - type: float64 - qtde de nan: 8137
> índice de avaliações

4. Coluna reviews_per_month - type: float64 - qtde de nan: 8137
> Avaliações por mês


Pela análise, todos os campos acima estão com valor NaN nos mesmos 8137 registros<br/>
A coluna 'number_of_reviews' está com o valor 0 (zero), logo, conclui-se que estes registros não possuem nenhuma avaliação e devem ser mantidos com NaN

### Colunas removidas
<table>
    <thead>
        <tr>
            <th>Coluna</th>
            <th>Tipo</th>
            <th>Dicionário</th>
            <th>Quantidade de NaN</th>
            <th>Análise</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>neighbourhood_group_cleansed</td>
            <td>float64</td>
            <td>Cálculo feito para a geolocalização da vizinhança</td>
            <td>31961/31961</td>
            <td>Outras infomações podem gerar este dado</td>
        </tr>
        <tr>
            <td>license</td>
            <td>float64</td>
            <td>Número de permissão/registro/licença</td>
            <td>31961/31961</td>
            <td>Número aparentemente não utilizado na legislação brasileira</td>
        </tr>
        <tr>
            <td>bathrooms</td>
            <td>float64</td>
            <td>Quantidade de banheiros</td>
            <td>31961/31961</td>
            <td>A informação pode constar na coluna "bathrooms_text"</td>
        </tr>
        <tr>
            <td>calendar_updated</td>
            <td>float64</td>
            <td>Sem informações</td>
            <td>31961/31961</td>
            <td>Coluna sem descrição e sem valor aparente</td>
        </tr>
        <tr>
            <td>neighbourhood</td>
            <td>Object</td>
            <td>Vizinhança do imóvel</td>
            <td>15223/31961</td>
            <td>Devido ao alto índice de NaN e ao tratamento já existente na coluna "neighbourhood_cleansed", será removida</td>
        </tr>
    </tbody>
</table>

In [9]:
df_listings.drop(['neighbourhood_group_cleansed'], axis=1, inplace=True)
df_listings.drop(['license'], axis=1, inplace=True)
df_listings.drop(['bathrooms'], axis=1, inplace=True)
df_listings.drop(['calendar_updated'], axis=1, inplace=True)
df_listings.drop(['neighbourhood'], axis=1, inplace=True)

### Conversões de tipos de dados
<table>
    <thead>
        <tr>
            <th>Coluna</th>
            <th>Tipo original</th>
            <th>Dicionário</th>
            <th>Novo tipo</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>last_scraped</td>
            <td>Object</td>
            <td>Data do webscraping</td>
            <td>datetime</td>
        </tr>
        <tr>
            <td>calendar_last_scraped</td>
            <td>Object</td>
            <td>Sem informações</td>
            <td>datetime</td>
        </tr>
        <tr>
            <td>first_review</td>
            <td>Object</td>
            <td>Data da primeira avaliação</td>
            <td>datetime</td>
        </tr>
        <tr>
            <td>last_review</td>
            <td>Object</td>
            <td>Data da última avaliação</td>
            <td>datetime</td>
        </tr>
        <tr>
            <td>host_has_profile_pic</td>
            <td>Object</td>
            <td>Imagem de perfil do anfitrião</td>
            <td>bool</td>
        </tr>
        <tr>
            <td>host_identity_verified</td>
            <td>Object</td>
            <td>Identidade verificada do anfitrião</td>
            <td>bool</td>
        </tr>
        <tr>
            <td>host_is_superhost</td>
            <td>Object</td>
            <td>Selo 'Superhost'</td>
            <td>bool</td>
        </tr>
        <tr>
            <td>has_availability</td>
            <td>Object</td>
            <td>Disponível</td>
            <td>bool</td>
        </tr>
        <tr>
            <td>instant_bookable</td>
            <td>Object</td>
            <td>Agendamento instantâneo</td>
            <td>bool</td>
        </tr>
        <tr>
            <td>host_acceptance_rate</td>
            <td>Object</td>
            <td>Índice de aceitação do anfitrião</td>
            <td>float</td>
        </tr>
        <tr>
            <td>host_listings_count</td>
            <td>float</td>
            <td>Número de imóveis do anfitrião</td>
            <td>int</td>
        </tr>
        <tr>
            <td>host_total_listings_count</td>
            <td>float</td>
            <td>Número de imóveis do anfitrião</td>
            <td>int</td>
        </tr>
        <tr>
            <td>beds</td>
            <td>float</td>
            <td>Quantidade de camas</td>
            <td>int</td>
        </tr>
        <tr>
            <td>bedrooms</td>
            <td>float</td>
            <td>Quantidade de quartos</td>
            <td>int</td>
        </tr>
    </tbody>
</table>

In [10]:
# object -> datetime
df_listings['last_scraped'] = pd.to_datetime(df_listings['last_scraped'], errors="coerce")
df_listings['calendar_last_scraped'] = pd.to_datetime(df_listings['calendar_last_scraped'], errors="coerce")
df_listings['first_review'] = pd.to_datetime(df_listings['first_review'], errors="coerce")
df_listings['last_review'] = pd.to_datetime(df_listings['last_review'], errors="coerce")

In [11]:
# preenche as linhas com dados de datas inválidas (NaT) e com zero e depois None 
df_listings[['last_scraped', 'calendar_last_scraped', 'first_review', 'last_review']] = df_listings[['last_scraped', 'calendar_last_scraped', 'first_review', 'last_review']].fillna(0)
df_listings[['last_scraped', 'calendar_last_scraped', 'first_review', 'last_review']] = df_listings[['last_scraped', 'calendar_last_scraped', 'first_review', 'last_review']].replace(0, None)

In [12]:
# object -> bool
mapeamento = {'t': True, 'f': False}
df_listings['host_has_profile_pic'] = df_listings['host_has_profile_pic'].map(mapeamento)
df_listings['host_identity_verified'] = df_listings['host_identity_verified'].map(mapeamento)
df_listings['has_availability'] = df_listings['has_availability'].map(mapeamento)
df_listings['instant_bookable'] = df_listings['instant_bookable'].map(mapeamento)
df_listings['host_is_superhost'] = df_listings['host_is_superhost'].map(mapeamento)

In [13]:
# tipos numéricos
df_listings['host_acceptance_rate'] = df_listings['host_acceptance_rate'].str.removesuffix('%').astype('float32')/100
df_listings['host_response_rate'] = df_listings['host_response_rate'].str.removesuffix('%').astype('float32')/100
df_listings['host_listings_count'] = df_listings['host_listings_count'].astype('int16')
df_listings['host_total_listings_count'] = df_listings['host_total_listings_count'].astype('int16')
df_listings['beds'] = df_listings['beds'].astype('int16')
df_listings['bedrooms'] = df_listings['bedrooms'].astype('int16')
df_listings['price'] = df_listings['price'].str.removeprefix('$').str.replace(',', '').astype('float32')

### Tratamento de strings

| Coluna                	| unidecode | espaços no início e fim | maiúsculas    | regex HTML    | ' ' -> '_' 	| quebra de linha 	|
| ---	                    | :---:     | :---:	                  | :---:	      | :---:         | :---:       | :---:             |
| source                	| ✅        | ✅                     | ✅           | ❌            | ✅     	| ❌                |
| name                  	| ❌        | ✅                     | ❌           | ✅            | ❌     	| ✅                |
| description           	| ❌        | ✅                     | ❌           | ✅            | ❌     	| ✅                |
| neighborhood_overview 	| ❌        | ✅                     | ❌           | ✅            | ❌     	| ✅                |
| host_about             	| ❌        | ✅                     | ❌           | ✅            | ❌     	| ✅                |
| host_response_time       	| ✅        | ✅                     | ✅           | ❌            | ✅     	| ❌                |
| host_neighbourhood       	| ✅        | ✅                     | ✅           | ❌            | ✅     	| ❌                |
| neighbourhood_cleansed    | ✅        | ✅                     | ✅           | ❌            | ✅     	| ❌                |
| property_type             | ✅        | ✅                     | ✅           | ❌            | ✅     	| ❌                |
| room_type                 | ✅        | ✅                     | ✅           | ❌            | ✅     	| ❌                |

In [14]:
df_listings['source'] = df_listings['source'].apply(lambda x: clean_string(x))
df_listings['name'] = df_listings['name'].apply(lambda x: clean_string(x, True))
df_listings['description'] = df_listings['description'].apply(lambda x: clean_string(x, True))
df_listings['neighborhood_overview'] = df_listings['neighborhood_overview'].apply(lambda x: clean_string(x, True))
df_listings['host_about'] = df_listings['host_about'].apply(lambda x: clean_string(x, True))
df_listings['host_response_time'] = df_listings['host_response_time'].apply(lambda x: clean_string(x))
df_listings['host_neighbourhood'] = df_listings['host_neighbourhood'].apply(lambda x: clean_string(x))
df_listings['neighbourhood_cleansed'] = df_listings['neighbourhood_cleansed'].apply(lambda x: clean_string(x))
df_listings['property_type'] = df_listings['property_type'].apply(lambda x: clean_string(x))
df_listings['room_type'] = df_listings['room_type'].apply(lambda x: clean_string(x))

### Inserção no BD

In [15]:
db.post_data(table="listings", data=df_listings.to_dict(orient="records"), schema='trusted')